In [1]:
%load_ext autoreload
%reload_ext autoreload

%autoreload 2
%matplotlib inline
import tensorflow
from tensorflow.compat.v1.keras.backend import get_session
tensorflow.compat.v1.disable_v2_behavior()
import kerasAC 
from kerasAC.generators.tiledb_predict_generator import *
from kerasAC.tiledb_config import *
import tiledb 
from scipy.special import softmax
from kerasAC.interpret.deepshap import * 
from kerasAC.interpret.profile_shap import * 
from kerasAC.vis import * 

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.
/users/annashch/kerasAC/kerasAC/vis/plot_letters.py:173: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  min_coords = np.vstack(data.min(0) for data in polygons_data).min(0)
/users/annashch/kerasAC/kerasAC/vis/plot_letters.py:174: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  max_coords = np.vstack(data.max(0) for data in polygons_data).max(0)


In [2]:
gen=TiledbPredictGenerator(ref_fasta="/mnt/data/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta",
                    batch_size=1,
                    bed_regions_summit_center=True,
                    bed_regions="/srv/scratch/annashch/bias_correction/enzymatic_bias/profile_model_of_bias/dnase/k562.chr1.bed",
                    tdb_array="/srv/scratch/annashch/bias_correction/enzymatic_bias/tiledb_inputs/merged.SRR1565781.SRR1565782",
                    chrom_sizes="/users/annashch/hg38.chrom.sizes",
                    tdb_input_flank=[673],
                    tdb_partition_attribute_for_upsample=[None],
                    tdb_input_source_attribute=["seq"],
                    tdb_partition_thresh_for_upsample=None,
                    tdb_input_aggregation=[None],
                    tdb_input_transformation=[None],
                    tdb_output_source_attribute=["bigwig_track","bigwig_track"],
                    tdb_output_flank=[500,500],
                    tdb_output_aggregation=[None,"sum"],
                    tdb_output_transformation=[None,"log"],
                    num_inputs=1,
                    num_outputs=2,
                    upsample_ratio=None,
                    tasks=['SRR1565781.SRR1565782'],
                    tdb_ambig_attribute="ambig_peak",
                    tdb_config=get_default_config(),
                    tdb_ctx=tiledb.Ctx(config=get_default_config()),
                    num_threads=1)
                    

opening:/srv/scratch/annashch/bias_correction/enzymatic_bias/tiledb_inputs/merged.SRR1565781.SRR1565782 for reading...
success!
got indices for used chroms
identified task indices:[0]
loaded bed regions
created generator
created predict generator


In [3]:
#load the model! 
from keras.models import load_model
from keras.utils.generic_utils import get_custom_objects
from kerasAC.metrics import * 
from kerasAC.custom_losses import * 
custom_objects={"recall":recall,
                    "sensitivity":recall,
                    "specificity":specificity,
                    "fpr":fpr,
                    "fnr":fnr,
                    "precision":precision,
                    "f1":f1,
                    "ambig_binary_crossentropy":ambig_binary_crossentropy,
                    "ambig_mean_absolute_error":ambig_mean_absolute_error,
                    "ambig_mean_squared_error":ambig_mean_squared_error,
                    "MultichannelMultinomialNLL":MultichannelMultinomialNLL}
get_custom_objects().update(custom_objects)


In [4]:
model=load_model("/srv/scratch/annashch/bias_correction/enzymatic_bias/profile_model_of_bias/dnase/bias.dnase.0.hdf5")
model_wrapper=(model.input, model.layers[-1].output)
count_explainer=shap.DeepExplainer(model_wrapper,data=create_background,combine_mult_and_diffref=combine_mult_and_diffref_1d)
prof_explainer = create_explainer(model)
all_coords=[] 
all_profile_explanations=[] 
all_count_explanations=[] 
for i in range(100): 
    X,y,coords=gen[i]
    coords=coords[0]
    profile_explanations=prof_explainer(X[0],None,None)
    count_explanations=np.squeeze(count_explainer.shap_values(X)[0])
    all_coords.append(coords)
    all_profile_explanations.append(profile_explanations)
    all_count_explanations.append(count_explanations)
    if i%50==0: 
        print(str(i))
        

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
HERE--tf
HERE--keras

here!
here!


0
50


In [5]:
import pickle
outputs={} 
outputs['coords']=all_coords
outputs['shap_profile']=all_profile_explanations
outputs['shap_counts']=all_count_explanations
pickle.dump( outputs,open( "k562.small.deepshap.bpnet.p", "wb" ) )
